Build Full Journal Database
===

This should probably be converted into a standalone script.

It builds a database table from the contents of the flattened journal JSON file.

All columns are included with the appropriate types, with the exception of `siteId` and `_id_$oid` which are transformed into the columns `site_id` and `journal_oid` respectively.

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
from IPython.core.display import display, HTML

import os
import numpy as np
import pandas as pd
import itertools

import matplotlib.pyplot as plt
import matplotlib.dates as md
import matplotlib
import pylab as pl

import datetime as dt
import time

from collections import Counter

import json
import os
import re
from html.parser import HTMLParser
import itertools
import multiprocessing as mp
from nltk import word_tokenize
from IPython.core.display import display, HTML
import datetime as dt

from tqdm import tqdm

import collections
import sqlite3

In [2]:
flattened_json_field_counts_filename = "/home/srivbane/levon003/repos/misc-scripts/json_field_counts/journal_flat_field_counts.json"
assert os.path.exists(flattened_json_field_counts_filename)

working_dir = "/home/srivbane/shared/caringbridge/data/projects/qual-health-journeys/extract_site_features"
flattened_journal_json_filename = os.path.join(working_dir, "journal_flat.json")

journal_db_filename = os.path.join(working_dir, "journal.db")

In [3]:
#Build a database connection for the jouranl
def get_db():
    db_filename = journal_db_filename
    db = sqlite3.connect(
            db_filename,
            detect_types=sqlite3.PARSE_DECLTYPES
        )
    db.row_factory = sqlite3.Row
    return db

In [4]:
with open(flattened_json_field_counts_filename, 'r') as infile:
    count_data = " ".join(infile.readlines())
    field_count_dict = eval(count_data)
field_count_dict

{'_id_$oid_str_count': 15327592,
 'amps_count_int_count': 14096064,
 'amps_str_count': 14096064,
 'body_str_count': 15301485,
 'cm_dah_int_count': 1582385,
 'cm_daw_int_count': 1582385,
 'cm_dh_int_count': 1582385,
 'cm_dpi_int_count': 1581882,
 'cm_dw_int_count': 1582385,
 'cm_touch_str_count': 1582170,
 'cm_vph_int_count': 1582385,
 'cm_vpw_int_count': 1582385,
 'createdAt_int_count': 15327592,
 'draft_body_str_count': 157009,
 'draft_id_str_count': 1439163,
 'draft_title_str_count': 156654,
 'ip_str_count': 11833756,
 'isDeleted_str_count': 1348696,
 'isDraft_NoneType_count': 2968395,
 'isDraft_str_count': 12092282,
 'isPriority_str_count': 76981,
 'journalId_int_count': 11833756,
 'lastEdit_int_count': 2196461,
 'pendingPublish_NoneType_count': 2968422,
 'pendingPublish_str_count': 110624,
 'photos_caption_str_count': 15327592,
 'photos_count_int_count': 15327592,
 'photos_cropped_str_count': 15327592,
 'photos_heights_str_count': 15327592,
 'photos_widths_str_count': 15327592,
 'p

In [5]:
def get_sql_type(python_type_str):
    if python_type_str == "str":
        sql_type = "TEXT"
    elif python_type_str == "int":
        sql_type = "INTEGER"
    else:
        raise ValueError(python_type_str + " not valid.")
    return sql_type

omit_fields = ["siteId", "_id_$oid"]
field_type_list = []
for key in field_count_dict.keys():
    tokens = key.split("_")
    field_name = "_".join(tokens[:-2])
    field_python_type = tokens[-2]
    if field_name in omit_fields:
        continue
    if field_python_type == "NoneType":
        continue
    #print(field_name, field_python_type)
    field_type_list.append((field_name, get_sql_type(field_python_type)))
field_type_list

[('amps_count', 'INTEGER'),
 ('amps', 'TEXT'),
 ('body', 'TEXT'),
 ('cm_dah', 'INTEGER'),
 ('cm_daw', 'INTEGER'),
 ('cm_dh', 'INTEGER'),
 ('cm_dpi', 'INTEGER'),
 ('cm_dw', 'INTEGER'),
 ('cm_touch', 'TEXT'),
 ('cm_vph', 'INTEGER'),
 ('cm_vpw', 'INTEGER'),
 ('createdAt', 'INTEGER'),
 ('draft_body', 'TEXT'),
 ('draft_id', 'TEXT'),
 ('draft_title', 'TEXT'),
 ('ip', 'TEXT'),
 ('isDeleted', 'TEXT'),
 ('isDraft', 'TEXT'),
 ('isPriority', 'TEXT'),
 ('journalId', 'INTEGER'),
 ('lastEdit', 'INTEGER'),
 ('pendingPublish', 'TEXT'),
 ('photos_caption', 'TEXT'),
 ('photos_count', 'INTEGER'),
 ('photos_cropped', 'TEXT'),
 ('photos_heights', 'TEXT'),
 ('photos_widths', 'TEXT'),
 ('platform', 'TEXT'),
 ('promo', 'TEXT'),
 ('publishedAt', 'INTEGER'),
 ('replies_count', 'INTEGER'),
 ('replies', 'TEXT'),
 ('title', 'TEXT'),
 ('updatedAt', 'INTEGER'),
 ('userId', 'INTEGER'),
 ('videos_count', 'INTEGER'),
 ('videos', 'TEXT')]

In [6]:
field_names = [field_name for field_name, field_type in field_type_list]
field_names

['amps_count',
 'amps',
 'body',
 'cm_dah',
 'cm_daw',
 'cm_dh',
 'cm_dpi',
 'cm_dw',
 'cm_touch',
 'cm_vph',
 'cm_vpw',
 'createdAt',
 'draft_body',
 'draft_id',
 'draft_title',
 'ip',
 'isDeleted',
 'isDraft',
 'isPriority',
 'journalId',
 'lastEdit',
 'pendingPublish',
 'photos_caption',
 'photos_count',
 'photos_cropped',
 'photos_heights',
 'photos_widths',
 'platform',
 'promo',
 'publishedAt',
 'replies_count',
 'replies',
 'title',
 'updatedAt',
 'userId',
 'videos_count',
 'videos']

In [7]:
create_field_strings = [ "{} {}".format(field_name, field_type)
    for field_name, field_type in field_type_list
]
create_field_string = ",\n".join(create_field_strings)

create_table_command = """
CREATE TABLE journal (
id INTEGER PRIMARY KEY AUTOINCREMENT,
site_id INTEGER NOT NULL,
journal_oid TEXT NOT NULL,
%s
)
""" % create_field_string
print(create_table_command)


CREATE TABLE journal (
id INTEGER PRIMARY KEY AUTOINCREMENT,
site_id INTEGER NOT NULL,
journal_oid TEXT NOT NULL,
amps_count INTEGER,
amps TEXT,
body TEXT,
cm_dah INTEGER,
cm_daw INTEGER,
cm_dh INTEGER,
cm_dpi INTEGER,
cm_dw INTEGER,
cm_touch TEXT,
cm_vph INTEGER,
cm_vpw INTEGER,
createdAt INTEGER,
draft_body TEXT,
draft_id TEXT,
draft_title TEXT,
ip TEXT,
isDeleted TEXT,
isDraft TEXT,
isPriority TEXT,
journalId INTEGER,
lastEdit INTEGER,
pendingPublish TEXT,
photos_caption TEXT,
photos_count INTEGER,
photos_cropped TEXT,
photos_heights TEXT,
photos_widths TEXT,
platform TEXT,
promo TEXT,
publishedAt INTEGER,
replies_count INTEGER,
replies TEXT,
title TEXT,
updatedAt INTEGER,
userId INTEGER,
videos_count INTEGER,
videos TEXT
)



In [8]:
# create the table
try:
    db = get_db()
    db.execute("DROP TABLE IF EXISTS journal")
    db.execute(create_table_command)
finally:
    db.close()

In [9]:
column_qmarks = "(%s)" % ",".join("?"* (len(field_names) + 2))
column_names = "(site_id,journal_oid,%s)" % ",".join(field_names)
print(column_names)
print(column_qmarks)
insert_command = 'INSERT INTO journal %s VALUES %s' % (column_names, column_qmarks)
print(insert_command)

(site_id,journal_oid,amps_count,amps,body,cm_dah,cm_daw,cm_dh,cm_dpi,cm_dw,cm_touch,cm_vph,cm_vpw,createdAt,draft_body,draft_id,draft_title,ip,isDeleted,isDraft,isPriority,journalId,lastEdit,pendingPublish,photos_caption,photos_count,photos_cropped,photos_heights,photos_widths,platform,promo,publishedAt,replies_count,replies,title,updatedAt,userId,videos_count,videos)
(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)
INSERT INTO journal (site_id,journal_oid,amps_count,amps,body,cm_dah,cm_daw,cm_dh,cm_dpi,cm_dw,cm_touch,cm_vph,cm_vpw,createdAt,draft_body,draft_id,draft_title,ip,isDeleted,isDraft,isPriority,journalId,lastEdit,pendingPublish,photos_caption,photos_count,photos_cropped,photos_heights,photos_widths,platform,promo,publishedAt,replies_count,replies,title,updatedAt,userId,videos_count,videos) VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)


In [10]:
try:
    db = get_db()
    
    TOTAL_TRANSACTIONS = 100
    COMMIT_INTERVAL = int(15327592 / TOTAL_TRANSACTIONS)
    print("Commiting transaction every %d inserts." % COMMIT_INTERVAL)
    transaction_count = 1
    
    with open(flattened_journal_json_filename, encoding="utf-8") as infile:
        db.execute('BEGIN TRANSACTION')
        for i, line in tqdm(enumerate(infile), total=15327592):
            journal = json.loads(line)
            site_id = int(journal["siteId"])
            journal_oid = journal["_id_$oid"]
            
            col_values = [journal[field_name] if field_name in journal else None
                for field_name in field_names
            ]
            values = (site_id, journal_oid, *col_values)
            
            db.execute(insert_command, values)
            
            if (i+1) % COMMIT_INTERVAL == 0:
                db.commit()
                db.execute('BEGIN TRANSACTION')
                transaction_count += 1
                print("Beginning transaction %d / %d." % (transaction_count, TOTAL_TRANSACTIONS))
        db.commit()
finally:
    db.close()

  0%|          | 1100/15327592 [00:00<23:13, 10995.03it/s]

Commiting transaction every 153275 inserts.


  1%|          | 154563/15327592 [00:18<29:51, 8467.51it/s]

Beginning transaction 2 / 100.


  2%|▏         | 308247/15327592 [00:34<27:56, 8956.30it/s]

...

Beginning transaction 99 / 100.


 99%|█████████▉| 15175019/15327592 [31:12<00:18, 8105.34it/s]

Beginning transaction 100 / 100.


100%|██████████| 15327592/15327592 [31:40<00:00, 8065.25it/s]

Beginning transaction 101 / 100.


In [16]:
try:
    db = get_db()
    count_command = """
    SELECT COUNT(1) FROM journal
    """
    count_row = db.execute(count_command).fetchone()
    print(count_row.keys())
finally:
    db.close()

['COUNT(1)']


In [18]:
assert count_row['COUNT(1)'] == 15327592

In [15]:
try:
    db = get_db()
    index_command = """
    CREATE INDEX journal_siteId_journalOid ON journal (site_id, journal_oid);
    """
    db.execute(index_command)
finally:
    db.close()